<a href="https://colab.research.google.com/github/uumami/mhar_pytorch/blob/master/mhar_mst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}\nsetInterval(ClickConnect,60000)\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cat /proc/meminfo

MemTotal:       26751684 kB
MemFree:        22290432 kB
MemAvailable:   25693160 kB
Buffers:          132224 kB
Cached:          3468252 kB
SwapCached:            0 kB
Active:          1027984 kB
Inactive:        3065848 kB
Active(anon):     454692 kB
Inactive(anon):      376 kB
Active(file):     573292 kB
Inactive(file):  3065472 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               876 kB
Writeback:             0 kB
AnonPages:        493488 kB
Mapped:           259216 kB
Shmem:               928 kB
Slab:             230896 kB
SReclaimable:     173924 kB
SUnreclaim:        56972 kB
KernelStack:        4712 kB
PageTables:         6208 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375840 kB
Committed_AS:    3210104 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1888 kB
AnonHugePages:   

In [ ]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("GSAR")

R[write to console]: Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

R[write to console]: Installing package(s) 'GSAR'

R[write to console]: trying URL 'https://bioconductor.org/packages/3.10/bioc/src/contrib/GSAR_1.20.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2185168 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

In [ ]:
%%R
library(tidyverse)
library(MASS)
library(GSAR)
library(parallel)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.2     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: 
Attaching package: ‘MASS’


R[write to console]: The following object is masked from ‘package:dplyr’:

    select


R[write to console]: Loading required package: igraph

R[write to console]: 
Attaching package: ‘igraph’


R[write to console]: The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


R[write to console]: The following objects are masked from ‘package:purrr’:

    compose, simplify


R[write to console]: The following object is masked from ‘package:tidyr’:

    crossing


R[w

In [ ]:
%%R

# Simplex sample generator
simplex_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n-1,0,1)  %>%  
                    sort(decreasing=FALSE)
        sample  <- append(0.0, sample)  %>% 
                    append(1.0)
        sample  <- sample[-1] - sample
        sample  <- sample[1:n]

        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# Genberate hypercube
hyper_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n,-1,1) 
        sample  <- sample[1:n]
        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# MST for simplex
mst <- function(file_, nsamp=10000, perm=50){
    # Read files with samples
    df_  <- read_csv(file_)  %>%  
    dplyr::select(starts_with('x_'))  %>% 
    sample_n(size=nsamp, replace=FALSE)
    # Get dim of the read file
    n_rows  <- dim(df_)[1]
    n_cols  <- dim(df_)[2]
        if (n_rows > (nsamp-1)){
        simplex_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
        
        df_  <- df_  %>% 
                data.matrix()   %>% 
                unname()
        
        df_  <- rbind(df_, simplex_sample)  
        df_  <- df_ %>% 
                aperm(c(2,1)) 
        result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                         nperm=perm, pvalue.only=FALSE)
        z_df  <- tibble(dim=n_cols, z=result$statistic, 
                        p_value=result$p.value,
                             nsamp=nsamp)
        gc()
        return(z_df)
        
    }
    
}


In [ ]:
%%R
path  <- '/drive/My Drive/mhar_test_times_p100/'
set.seed(5)
simplex_gen(5,10)
files <- list.files(path)
files  <- paste(path, files, sep='')

# Extract dimenions
m <- regexpr("%[0-9]+.csv", files, perl=TRUE)
dimension <- regmatches(files, m)
m <- regexpr("[0-9]+", dimension, perl=TRUE)
dimension <- regmatches(dimension, m)
dimension

#Extract figure
m <- regexpr("[0-9]+_[a-z]+%", files, perl=TRUE)
figure <- regmatches(files, m)
m <- regexpr("[a-z]+", figure, perl=TRUE)
figure <- regmatches(figure, m)
figure

# Extract seed
m <- regexpr("/[0-9]+_", files, perl=TRUE)
sed <- regmatches(files, m)
m <- regexpr("[0-9]+", sed, perl=TRUE)
sed <- regmatches(sed, m)

df_simulations <- tibble(file=files, n=dimension, figure=figure, seed=sed)

In [ ]:
%%R

save_path <- '/content/drive/My Drive/mst_5000/mst_7000.csv'

# Create random seeds
seeds_ <- sample(1:10000000, 1000, replace=T)
s<- 1


## PArse existing experiments
if(!file.exists(save_path)){
    existance<- tibble(figure=character(),dim=numeric(), z=numeric(), 
                          p_value=numeric(),
                              nsamp=numeric(), file=character(), perm=numeric(),
               seed=numeric()) 
    
}else{
    existance <- read.csv(save_path) %>% dplyr::select( -contains('X'))
}

z_df <- existance

for(f in files){
  # Set hyperparameters
  set.seed(seeds_[s])
  file_name <- f
  perm <- 50
  n_sample <-7000
  print(file_name)
  df_  <- read_csv(file_name)
  df_ <- df_[c(1:n_sample),]
  # Get dim of the read file
  n_rows  <- dim(df_)[1]
  n_cols  <- dim(df_)[2]
  print(n_rows)
  print(n_cols)

  # Check if the experiment already runned
  check_df <- FALSE
  if(f %in% existance$f){
  check_df <- existance %>% filter(file==f)
  check_df <- (check_df$dim == n_cols) && (check_df$nsamp == n_sample)  && (check_df$perm == perm)
  }

  check_df <- !check_df

  if(check_df){
    if (grepl('hyper', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
    typ <- 'Hypercube'
  }
  if (grepl('simplex', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
    typ <- 'Simplex'
  }

  df_  <- df_  %>% data.matrix()   %>% unname()
          
  df_  <- rbind(df_, real_sample)  
  df_  <- df_ %>% aperm(c(2,1)) 
  dim(df_)
  result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                          nperm=2, pvalue.only=FALSE)
  z_df  <- tibble(figure=typ,dim=n_cols, z=result$statistic, 
                          p_value=result$p.value,
                              nsamp=n_sample, file=f, perm=perm, 
                  seed=seeds_[s]) %>% rbind(z_df)
  z_df %>% write.csv(save_path)
  gc()  
  
  }
  #Increment seed index
  s <- s + 1 
}




[1] "/content/drive/My Drive/mhar_paper_samples_2/100_hypercube%1000%3.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double()
)



[1] 7000
[1] 3
[1] "/content/drive/My Drive/mhar_paper_samples_2/1064_hypercube%1000%15.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double(),
  `10` = col_double(),
  `11` = col_double(),
  `12` = col_double(),
  `13` = col_double(),
  `14` = col_double()
)



[1] 7000
[1] 15
[1] "/content/drive/My Drive/mhar_paper_samples_2/1064_hypercube%1000%25.csv"


...

[1] 7000
[1] 50
[1] "/content/drive/My Drive/mhar_paper_samples_2/133_hypercube%1000%15.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double(),
  `10` = col_double(),
  `11` = col_double(),
  `12` = col_double(),
  `13` = col_double(),
  `14` = col_double()
)



[1] 7000
[1] 15
[1] "/content/drive/My Drive/mhar_paper_samples_2/133_hypercube%1000%25.csv"


R[write to console]: Parsed with column specification:
cols(
  .default = col_double()
)

R[write to console]: See spec(...) for full column specifications.



[1] 7000
[1] 25
